# DHT22 Prediction Model

아래 중간 중간 `#빈칸`을 채우며 적절히 설계해보세요. 끝까지 실행하면 `temp_predict.onnx` 및 `humi_predict.onnx` 파일이 생성됩니다.

In [1]:
import json
import torch
import torch.nn as nn
import torch.onnx
import numpy as np
from torch.utils.data import DataLoader, Dataset

### Class Definitions

In [2]:
class DHTDataset(Dataset):
    def __init__(self, key):
        self.data = []
        self.target = []
        self.key = key

        db = open("db.json", "r")
        humi_temp = json.load(db)

        for i in range(len(humi_temp.get(self.key))-62):
            tmp_data = []
            tmp_data.append([float(humi_temp.get(self.key)[i][1])])
            tmp_data.append([float(humi_temp.get(self.key)[i+1][1])])
            tmp_data.append([float(humi_temp.get(self.key)[i+2][1])])
            self.data.append(torch.as_tensor(tmp_data))

            self.target.append(
                torch.as_tensor([float(humi_temp.get(self.key)[i+62][1])]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data, target = self.data[index], self.target[index]
        return data, target

In [29]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()

        self.linear = nn.Sequential(
           nn.Linear(3,1)
        )
        

    def forward(self, x):
        x = self.linear(x.squeeze())
        return x

### Function Definitions

##### A function to split dataset into several data loaders

In [30]:
def splitDataset(dataset):
    test_size = int(len(dataset)/6)
    val_size = int(len(dataset)/6)
    train_size = len(dataset) - test_size - val_size

    trainset, valset, testset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size])

    trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
    validloader = DataLoader(valset, batch_size=16, shuffle=True)
    testloader = DataLoader(testset, batch_size=16, shuffle=True)

    return trainloader, validloader, testloader

##### A function to train model

In [31]:
def train_model(model, patience, num_epochs, train_loader, valid_loader):
    train_losses = []
    valid_losses = []
    mean_train_losses = []
    mean_valid_losses = []
    p = 0
    min_valid_loss = float("inf")

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1, num_epochs + 1):

        model.train()
        for train_data, train_target in train_loader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                train_data = train_data.to(device, dtype=torch.float)
                train_target = train_target.to(device, dtype=torch.float)
            optimizer.zero_grad()
            output = model(train_data)
            loss = criterion(output, train_target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()

        for valid_data, valid_target in valid_loader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                valid_data = valid_data.to(device, dtype=torch.float)
                valid_target = valid_target.to(device, dtype=torch.float)
            output = model(valid_data)
            loss = criterion(output, valid_target)
            valid_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        valid_loss = np.mean(valid_losses)
        mean_train_losses.append(train_loss)
        mean_valid_losses.append(valid_loss)

        if min_valid_loss > valid_loss:
            min_valid_loss = valid_loss
            print(f"min_valid_loss: {min_valid_loss}")

        epoch_len = len(str(num_epochs))
        print_msg = (f"[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] " +
                     f"train_loss: {train_loss:.5f} " +
                     f"valid_loss: {valid_loss:.5f} ")
        print(print_msg)

        train_losses = []
        valid_losses = []

        if min_valid_loss < valid_loss and epoch > 1:
            p = p + 1
            print(f'patience: {p}')
        else:
            p = 0
            torch.save(model.state_dict(), "bestmodel.pt")
            print("Saving Model...")

        if patience == p:
            print("Early stopping")
            break

    model.load_state_dict(torch.load("bestmodel.pt"))

    return model

### Training

##### Load datasets and get dataloaders

In [32]:
tempDataset = DHTDataset("temperature")
humiDataset = DHTDataset("humidity")

temp_trainloader, temp_validloader, temp_testloader = splitDataset(tempDataset)
humi_trainloader, humi_validloader, humi_testloader = splitDataset(humiDataset)

##### Make models

In [33]:
tempModel = SimpleModel()
humiModel = SimpleModel()

if torch.cuda.is_available():
    tempModel = tempModel.cuda()
    humiModel = humiModel.cuda()

##### Do training with earlystopping

In [34]:
temp_predict_model = train_model(tempModel, patience=30, num_epochs=1000,
                                 train_loader=temp_trainloader,
                                 valid_loader=temp_validloader)
torch.save(temp_predict_model.state_dict(), "temp_predict.pt")

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


min_valid_loss: 2.6229708796148903
[   1/1000] train_loss: 123.47335 valid_loss: 2.62297 
Saving Model...
min_valid_loss: 0.003266205022563562
[   2/1000] train_loss: 0.32901 valid_loss: 0.00327 
Saving Model...
min_valid_loss: 0.0031735118757112374
[   3/1000] train_loss: 0.00348 valid_loss: 0.00317 
Saving Model...
[   4/1000] train_loss: 0.00346 valid_loss: 0.00318 
patience: 1
[   5/1000] train_loss: 0.00346 valid_loss: 0.00321 
patience: 2
[   6/1000] train_loss: 0.00346 valid_loss: 0.00320 
patience: 3
[   7/1000] train_loss: 0.00346 valid_loss: 0.00318 
patience: 4
[   8/1000] train_loss: 0.00348 valid_loss: 0.00317 
patience: 5
[   9/1000] train_loss: 0.00346 valid_loss: 0.00319 
patience: 6
[  10/1000] train_loss: 0.00348 valid_loss: 0.00318 
patience: 7
[  11/1000] train_loss: 0.00347 valid_loss: 0.00321 
patience: 8
[  12/1000] train_loss: 0.00347 valid_loss: 0.00322 
patience: 9
[  13/1000] train_loss: 0.00348 valid_loss: 0.00318 
patience: 10
min_valid_loss: 0.003154915492

In [35]:
humi_predict_model = train_model(humiModel, patience=30, num_epochs=1000,
                                 train_loader=humi_trainloader,
                                 valid_loader=humi_validloader)
torch.save(humi_predict_model.state_dict(), "humi_predict.pt")

min_valid_loss: 142.12180486455694
[   1/1000] train_loss: 987.97827 valid_loss: 142.12180 
Saving Model...
min_valid_loss: 1.0482591181188017
[   2/1000] train_loss: 33.04656 valid_loss: 1.04826 
Saving Model...
min_valid_loss: 0.07599061453161207
[   3/1000] train_loss: 0.39506 valid_loss: 0.07599 
Saving Model...
min_valid_loss: 0.07506937923765666
[   4/1000] train_loss: 0.24641 valid_loss: 0.07507 
Saving Model...
min_valid_loss: 0.0750012493876925
[   5/1000] train_loss: 0.24641 valid_loss: 0.07500 
Saving Model...
[   6/1000] train_loss: 0.24642 valid_loss: 0.07500 
patience: 1
min_valid_loss: 0.07495552716572346
[   7/1000] train_loss: 0.24651 valid_loss: 0.07496 
Saving Model...
[   8/1000] train_loss: 0.24656 valid_loss: 0.07608 
patience: 1
[   9/1000] train_loss: 0.24640 valid_loss: 0.07510 
patience: 2
[  10/1000] train_loss: 0.24684 valid_loss: 0.07544 
patience: 3
[  11/1000] train_loss: 0.24695 valid_loss: 0.07521 
patience: 4
[  12/1000] train_loss: 0.24704 valid_loss:

### ONNX Coversion

In [36]:
def convert_to_onnx(saved_file_name):
    saved_model = SimpleModel()
    saved_model.load_state_dict(torch.load(saved_file_name))
    saved_model.eval()
    torch.onnx.export(
        saved_model,
        torch.randn((1, 3, 1)),
        saved_file_name[:-3] + '.onnx',
        opset_version=11,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"]
    )

In [37]:
convert_to_onnx("temp_predict.pt")
convert_to_onnx("humi_predict.pt")

In [38]:
# End of Document.